# Duck Duck Diameter

This is in response to Ben Freeman's Problem "Duck Pond Chances" posted on brilliant.org: https://brilliant.org/problems/duck-pond-chances/

Since Stochastic can sometimes be hard to grasp and I've seen some confusion in the discussion, I'd like to take a different approach.

We'll first transform the problem to make it less complex and easier to simulate, by transforming it to a one-dimensional question. Then we are going to develop an algorithm to simulate the problem and gather experimental evidence by repeating it a large number of times. Finally comparing our results to Indraneel Mukhopadhyaya's answer.



## Understanding and Transforming the problem

To quote the original problem:
> "4 small ducks are in a large circular pond, and can each be at any point in the circle with equal chance. What is the probability that a diameter can be drawn so that all 4 ducks are in the same semicircle of the pond?" - Ben Freeman

Our assumptions:
1. The ducks are small enough to neglect their size.
2. If a duck is exactly on the line, we consider it to be on both sides at the same time and allow a line to be drawn

When talking about a duck's position we can formulate it in terms of distance from the pond's center and an angle:

![Position in polar coordinates](img/positions.png)

This is also known as polar coordinates.

Since the distance from the center does not effect wheter a duck is within a specific half or not, we only have to use angles. This transformation can be geometrically visualized as projecting each duck to the circumference of the circle:

![Projected positions](img/projection.png)

Now we pick one of the ducks and cut the circumference on the opposite side, and unwrap it. This way we only have to worry about a line and not about a circle anymore. The distance between any two angles will be the same as before, so any distance on this line corresponds to an angle.

Cutting on the opposite side of a duck has the benefit of us not having to worry about wrapping around at the cut. Since the distance from the chosen duck to the cut is half the circumference and in terms of angles 180°.

![Cut and unwrap](img/cut.png)

We can now arbitrarily choose our units on the line to measure distances (corresponding to angles in the original problem). This could be from 
- 0 to 360, 
- 0 to 2 * pi or 
- simply 0 to 1. 

I prefer 0 to 1 because python's random produces random numbers between 0 and 1.

The question is now in relation to our line:

> Picking 4 equally likely points on a line segment of length 1. What is the propability that the distance between the outermost ducks is less then 0.5 (or 180°deg)?

This is now easy to calculate.

## Developing an algorithm 

Let's generate some random ducks on our line segment:

In [1]:
import random
random_ducks = [random.random() for _ in range(4)]
print(random_ducks)

[0.2493388064674913, 0.6385848666924715, 0.10423233467861481, 0.7819027016078891]


Now we'll adjust the values, so that the cut we made earlier is on the opposite side of one duck. 

While transforming the problem we first observed the positions and then made the cut. But by generating numbers between 0 and 1 we already made the cut. 

This is equivalent to a rotation of the pond around it's center (including the ducks).

In [2]:
rotation_delta = 0.5 - random_ducks[0];
rotated_ducks = [duck + rotation_delta for duck in random_ducks]
print(rotated_ducks)

[0.5, 0.8892460602249802, 0.3548935282111235, 1.032563895140398]


The first duck now sits at 0.5 (or in terms of angles this would be 180°deg).

Values above 1 and below 0 must now be adjusted to the other end of our strip. 
This is equivalent to the modulo 1 operation (in python this is written as `x % 1`):

In [3]:
ducks = [duck % 1 for duck in rotated_ducks]
print(ducks)

[0.5, 0.8892460602249802, 0.3548935282111235, 0.032563895140397925]


Now we sort them, so that the maximum distance (in terms of angles) is the difference between the largest (last) and the smallest (first) element

In [4]:
ducks.sort()
print(ducks)

[0.032563895140397925, 0.3548935282111235, 0.5, 0.8892460602249802]


And calculate the distance by distracting the first duck (ducks[0]) from the last duck (ducks[-1])

In [5]:
distance = ducks[-1] - ducks[0]

If this distance is equal or less than 0.5, than all ducks reside within the same half of our pond!

In [6]:
distance <= 0.5

False

## Experimental Analysis

When we repeat this experiment a sufficient number of times, the observed propability approaches the actual propability. So let's do this 100.000 times and see what happens:

In [7]:
def generate_ducks(duck_count):
    random_ducks = [random.random() for _ in range(duck_count)]
    rotation_delta = 0.5 - random_ducks[0]
    rotated_ducks = [duck + rotation_delta for duck in random_ducks]
    ducks = [duck % 1 for duck in rotated_ducks]
    ducks.sort()
    return ducks

def calculate_distance(ducks):
    return ducks[-1] - ducks[0]
    
def test_distance(distance):
    return distance <= 0.5
    
distances = []
hits = 0
iterations = 100000

for _ in range(iterations):
    ducks = generate_ducks(4)
    distance = calculate_distance(ducks)
    on_one_half = test_distance(distance)
    if on_one_half:
        hits += 1
        
print(hits, 'out of', iterations, 'times the ducks were on one side of the pool')
print('So our observed propability is', hits / iterations * 100, '%')

50071 out of 100000 times the ducks were on one side of the pool
So our observed propability is 50.071 %


This result strongly suggests that the correct answer to this question is indeed 1/2.

But what happens, if we change the number of ducks? And how does our experiment compare to Indraneel Mukhopadhyaya's answer prognosing the propability to be 
$$ n * \frac{1}{2^{n-1}}$$
where n is the number of ducks?

In [8]:
iterations = 50000

def test(duck_count):
    distances = []
    hits = 0    

    for _ in range(iterations):
        ducks = generate_ducks(duck_count)
        distance = calculate_distance(ducks)
        on_one_half = test_distance(distance)
        if on_one_half:
            hits += 1
    return (distances, hits, hits / iterations)

# Indraneel Mukhopadhyaya's answer to this problem formulates the propability as follows:
def expected_result(duck_count):
    return duck_count * (1 / 2 ** (duck_count-1))

for duck_count in [2,3,4,5,6,8,10,15,20]:
    distances, hits, propability = test(duck_count)
    expectation = expected_result(duck_count)
    print('for', duck_count, 'ducks we observed', hits, 'positive results (', propability * 100 , '% ) and we expected', expectation * 100, '%')

for 2 ducks we observed 50000 positive results ( 100.0 % ) and we expected 100.0 %
for 3 ducks we observed 37589 positive results ( 75.178 % ) and we expected 75.0 %
for 4 ducks we observed 25058 positive results ( 50.11600000000001 % ) and we expected 50.0 %
for 5 ducks we observed 15565 positive results ( 31.130000000000003 % ) and we expected 31.25 %
for 6 ducks we observed 9370 positive results ( 18.740000000000002 % ) and we expected 18.75 %
for 8 ducks we observed 3001 positive results ( 6.002 % ) and we expected 6.25 %
for 10 ducks we observed 973 positive results ( 1.9460000000000002 % ) and we expected 1.953125 %
for 15 ducks we observed 54 positive results ( 0.108 % ) and we expected 0.091552734375 %
for 20 ducks we observed 0 positive results ( 0.0 % ) and we expected 0.003814697265625 %


This lines up nicely to his answer. If you're not convinced, try playing around with more iterations, but keep in mind that it'll take longer to compute.

## Conclusion
The result of this experiment supports Mukhopadhyaya's answer. I wanted to show this approach as a method of eliminating any unnecessary complexity to reduce the problem to a more minimal one.